In [1]:
import gc
import dask_cudf as dcd

In [2]:
import pandas as pd

In [2]:
CUTOFF_START_DATE = "2024-03-01"
CUTOFF_END_DATE = "2024-04-01"

In [6]:
SEP = ','

date_cols = [
    "bus_date",
    "order_local_time",
    "ord_beg_time",
    "ord_close_time",
]

def concat_without_dups(keep: pd.DataFrame, drop: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([keep, drop[~drop["order_id"].isin(keep["order_id"])]])

### Fraud Label

In [4]:
%%time

label_dtype={'Appetize OrderId': 'object'}

satish_fraud = dcd.read_csv(
    's3://otg-prod-fraud-data/labels/2024_label/feb_jun_labels.csv', 
    dtype = label_dtype
).drop_duplicates().compute().to_pandas()

satish_fraud['Flo OrderID'] = satish_fraud['Flo OrderID'].astype(str)
satish_fraud['Flo OrderID'] = satish_fraud['Flo OrderID'].str.removeprefix('1')

satish_fraud['Flo OrderID'] = satish_fraud['Flo OrderID'].apply(
    lambda x: x.removesuffix('01') if len(x) == 10 else x)

satish_fraud['Appetize OrderId'] = satish_fraud['Appetize OrderId'].astype(str)
satish_fraud['Appetize OrderId'] = satish_fraud['Appetize OrderId'].str.removeprefix('1')
satish_fraud['Appetize OrderId'] = satish_fraud['Appetize OrderId'].apply(
    lambda x: x.removesuffix('01') if len(x) == 10 else x)

satish_fraud.drop_duplicates(subset= 'Flo OrderID', inplace = True) #'Appetize OrderId'

gc.collect()

CPU times: user 911 ms, sys: 185 ms, total: 1.1 s
Wall time: 1.23 s


179

### Order Table

In [8]:

order_dtypes = {
    'reduction_cd': 'object',
    'order_id': 'object',
    'original_order_id': 'object',
    'scheduled_local_dtm': 'object',
    'united_account_id': 'object',
    'employer_id': 'object',
    'employer_name': 'object',
    'identity_provider_id': 'object',
    'cust_prof_key': 'object',
    'cust_reduction_cd': 'object',
    'employee_role_name': 'object',
    'cust_active_fl': 'object',
    'cust_prof_id': 'object',
    'cust_emply_verified_fl': 'object',
    'cust_modified_time': 'object',
    'active_fl': 'object',
    'company_id': 'object',
    'default_email_address': 'object',
    'fee_amt': 'float64',
    'order_status_cd': 'object',
    'cust_create_time': 'object',
    'cust_first_name': 'object',
    'cust_last_name': 'object',
    'activity_status_code': 'object'
    }

# orders table
order_df = pd.read_csv(
            f"s3://otg-fraud-dataml-prod/v_order_details/daily/2024/10/7/lacm_000.csv",
            sep=SEP,
            dtype=order_dtypes,
            parse_dates=date_cols,
        )


order_df["order_id"] = order_df["order_id"].str.removeprefix("1")
# cust_orders["order_id"] = cust_orders["order_id"].str.removeprefix("14")

order_df["original_order_id"] = order_df["original_order_id"].str.removeprefix("1")
order_df["cust_prof_id"] = order_df["cust_prof_id"].str.removeprefix("14")

# Remove "01" suffix from `order_id`s with length 10.
cond = order_df["order_id"].str.len().eq(10)
ten_len_ords = order_df[cond]
ten_len_ords["order_id"] = ten_len_ords["order_id"].str.removesuffix("01")
order_df = concat_without_dups(order_df[~cond], ten_len_ords).drop_duplicates()


/tmp/ipykernel_1893/836952309.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ten_len_ords["order_id"] = ten_len_ords["order_id"].str.removesuffix("01")


### Payment Table

In [12]:


payment_dtypes = {
    'cash_recycler_tiny_code': 'object',
    'emp_email': 'object',
    'voucher_number': 'object',
    'account_id': 'object',
    'order_id': 'object'
    }

# Appetize payments
payment_df = pd.read_csv(
            f"s3://otg-fraud-dataml-prod/source_views/v_payment_details/daily/2024/10/7/lacm_000.csv",
            sep=SEP,
            dtype=payment_dtypes,
           # parse_dates=date_cols,
        )


payment_df["order_id"] = payment_df["order_id"].str.removeprefix("1")
# cust_payments["order_id"] = cust_payments["order_id"].str.removeprefix("14")

# Remove "01" suffix from `order_id`s with length 10.
cond = payment_df["order_id"].str.len().eq(10)
ten_len_ords = payment_df[cond]
ten_len_ords["order_id"] = ten_len_ords["order_id"].str.removesuffix("01")
payment_df = concat_without_dups(payment_df[~cond], ten_len_ords).drop_duplicates()


/tmp/ipykernel_1893/3144889484.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ten_len_ords["order_id"] = ten_len_ords["order_id"].str.removesuffix("01")


### Joining

In [13]:
data_cols = ['bus_date', 'pos_venue_id', 'order_id', 'device_order_id', 'time_zone',
       'order_local_time', 'sales_hr', 'ord_beg_time', 'ord_close_time',
       'item_count', 'gross_total', 'net_total', 'taxes', 'exclusive_tax',
       'inclusive_tax', 'reduction_amt', 'cust_prof_id', 'mobile_ord_fl',
       'original_order_id', 'vendor_loc_id', 'fee_amt', 'tip_amount',
       'sync_status', 'src_sys_id', 'vendor_id', 'menu_vendor_id',
       'refund_vendor_id', 'refund_venue_id', 'pos_terminal_id',
       'reduction_cd', 'active_fl', 'order_tab_id', 'order_status_cd',
       'united_account_id', 'card_type', 'payment_type', 'transaction_id',
       'transaction_seq_nu', 'cash_recycler_tiny_code', 'payment_amount',
       'account_id', 'payment_amt_rewards_points', 'voucher_number',
       'emp_email', 'cust_first_name', 'cust_last_name', 'birth_date',
       'activity_status_code', 'employee_role_id', 'employee_role_name',
       'cust_emply_verified_fl', 'cust_emply_active_fl', 'employer_id',
       'employer_name', 'employer_active_fl', 'default_email_address',
       'default_phone_number', 'company_id', 'primary_email_address',
       'primary_email_opt_in_fl', 'primary_email_active_fl',
       'primary_email_verified_fl', 'extra_ph_number',
       'is_fraud']

order_df = order_df.drop(order_df.columns.difference(data_cols), axis='columns')
payment_df = payment_df.drop(payment_df.columns.difference(data_cols), axis='columns')

In [31]:
reduction_cds = {
    "AIRPORT_DISCOUNT": "INT_AIRPORT_DISCOUNT",
    "Airport": "INT_AIRPORT_DISCOUNT",
    "AMERICAN_DISCOUNT": "INT_AMERICAN_DISCOUNT",
    "Chase": "INT_UNITED_CHASE",
    "CREW_DISCOUNT": "INT_CREW_DISCOUNT",
    "Crew": "INT_CREW_DISCOUNT",
    "DELTA_DISCOUNT": "INT_DELTA_DISCOUNT",
    "Delta": "INT_DELTA_DISCOUNT",
    "JETBLUE_DISCOUNT": "INT_JETBLUE_DISCOUNT",
    "MANAGER_DISCOUNT": "INT_MANAGER_DISCOUNT",
    "Manager Meal": "INT_MANAGER_DISCOUNT",
    "SOUTHWEST_DISCOUNT": "INT_SOUTHWEST_DISCOUNT",
    "SPIRIT_DISCOUNT": "INT_SPIRIT_DISCOUNT",
    "UNITED_CHASE": "INT_UNITED_CHASE",
    "UNITED_DISCOUNT": "INT_UNITED_DISCOUNT",
}
card_types = {
    "MC": "MASTERCARD",
    "AMEX": "AMERICAN EXPRESS",
    "DISC": "DISCOVER",
    "DISCOVER (DC)": "DISCOVER",
    "Invoice": "Invoice/ Military",
    "Military": "Invoice/ Military",
}

order_payment = (
    order_df.drop(["pos_venue_id", "transaction_id", "src_sys_id", "vendor_id", "active_fl"], axis="columns")
    .merge(
        payment_df.drop(["bus_date"], axis="columns"),
        on = "order_id",
        how="left",
        )
    )

order_payment["reduction_cd"] = order_payment["reduction_cd"].replace(to_replace=reduction_cds)
order_payment["card_type"] = order_payment["card_type"].replace(to_replace=card_types)


In [32]:
order_payment.describe()

,bus_date,order_local_time,sales_hr,ord_beg_time,ord_close_time,item_count,gross_total,net_total,taxes,exclusive_tax,...,primary_email_verified_fl,extra_ph_number,pos_venue_id,src_sys_id,transaction_id,vendor_id,transaction_seq_nu,payment_amount,payment_amt_rewards_points,active_fl
count,61726,61726,61726.000000,61726,61726,61726.000000,61317.000000,61726.000000,61557.000000,0.0,...,0.0,0.0,61726.000000,61726.0,0.0,61726.000000,61451.000000,61451.000000,96.000000,61726.0
mean,2024-01-01 00:00:00,2024-01-01 13:42:23.361565440,13.141448,2024-01-01 18:57:54.126559488,2024-01-01 18:57:54.126559488,2.230940,15.907647,16.928238,1.234603,NaN,...,NaN,NaN,1139.651654,1.0,NaN,11229.979360,1.012823,19.679134,4172.708333,1.0
min,2024-01-01 00:00:00,2024-01-01 03:00:54,0.000000,2024-01-01 08:00:54,2024-01-01 08:00:54,0.000000,0.000000,-34.970000,-2.980000,NaN,...,NaN,NaN,1109.000000,1.0,NaN,1804.000000,1.000000,-41.850000,670.000000,1.0
25%,2024-01-01 00:00:00,2024-01-01 09:35:33,9.000000,2024-01-01 14:56:40.249999872,2024-01-01 14:56:40.249999872,1.000000,5.940000,5.990000,0.350000,NaN,...,NaN,NaN,1140.000000,1.0,NaN,11261.000000,1.000000,6.520000,2037.500000,1.0
50%,2024-01-01 00:00:00,2024-01-01 14:09:21.500000,14.000000,2024-01-01 19:27:46,2024-01-01 19:27:46,2.000000,11.580000,12.000000,0.860000,NaN,...,NaN,NaN,1142.000000,1.0,NaN,11347.000000,1.000000,14.050000,3630.000000,1.0
75%,2024-01-01 00:00:00,2024-01-01 17:39:58.750000128,17.000000,2024-01-01 22:58:38.750000128,2024-01-01 22:58:38.750000128,3.000000,19.990000,20.980000,1.610000,NaN,...,NaN,NaN,1145.000000,1.0,NaN,11418.000000,1.000000,24.370000,5780.000000,1.0
max,2024-01-01 00:00:00,2024-01-02 02:59:25,23.000000,2024-01-02 08:57:42,2024-01-02 08:57:42,26.000000,208.230000,297.070000,26.370000,NaN,...,NaN,NaN,1156.000000,1.0,NaN,11611.000000,8.000000,376.910000,13800.000000,1.0
std,NaN,NaN,4.729429,NaN,NaN,1.700847,15.146113,16.724655,1.369320,NaN,...,NaN,NaN,10.293022,0.0,NaN,1057.138921,0.171903,19.926782,2727.998873,0.0


In [19]:
len(order_payment)

61726

In [28]:
our_df = pd.read_csv(
            f"s3://otg-fraud-dataml-prod/acm/daily/2024/10/7/acm_output.csv",
    sep=',')

In [29]:
len(our_df)

61726

In [30]:
our_df.describe()

,order_id,sales_hr,item_count,gross_total,net_total,taxes,exclusive_tax,inclusive_tax,reduction_amt,cust_prof_id,...,extra_ph_number,pos_venue_id,src_sys_id,transaction_id,vendor_id,transaction_seq_nu,payment_amount,payment_amt_rewards_points,voucher_number,active_fl
count,6.172600e+04,61726.000000,61726.000000,61317.000000,61726.000000,61557.000000,0.0,0.0,61725.000000,1.446400e+04,...,0.0,61726.000000,61726.0,0.0,61726.000000,61451.000000,61451.000000,96.000000,1.000000e+01,61726.0
mean,4.003846e+07,13.141448,2.230940,15.907647,16.928238,1.234603,NaN,NaN,-0.221015,1.082653e+07,...,NaN,1139.651654,1.0,NaN,11229.979360,1.012823,19.679134,4172.708333,2.233607e+08,1.0
std,1.737045e+04,4.729429,1.700847,15.146113,16.724655,1.369320,NaN,NaN,1.695433,1.463293e+06,...,NaN,10.293022,0.0,NaN,1057.138921,0.171903,19.926782,2727.998873,3.549397e+01,0.0
min,4.000854e+07,0.000000,0.000000,0.000000,-34.970000,-2.980000,NaN,NaN,-83.470000,4.690000e+03,...,NaN,1109.000000,1.0,NaN,1804.000000,1.000000,-41.850000,670.000000,2.233606e+08,1.0
25%,4.002344e+07,9.000000,1.000000,5.940000,5.990000,0.350000,NaN,NaN,0.000000,1.111597e+07,...,NaN,1140.000000,1.0,NaN,11261.000000,1.000000,6.520000,2037.500000,2.233607e+08,1.0
50%,4.003839e+07,14.000000,2.000000,11.580000,12.000000,0.860000,NaN,NaN,0.000000,1.111908e+07,...,NaN,1142.000000,1.0,NaN,11347.000000,1.000000,14.050000,3630.000000,2.233607e+08,1.0
75%,4.005341e+07,17.000000,3.000000,19.990000,20.980000,1.610000,NaN,NaN,0.000000,1.112221e+07,...,NaN,1145.000000,1.0,NaN,11418.000000,1.000000,24.370000,5780.000000,2.233607e+08,1.0
max,4.015117e+07,23.000000,26.000000,208.230000,297.070000,26.370000,NaN,NaN,10.490000,1.112556e+07,...,NaN,1156.000000,1.0,NaN,11611.000000,8.000000,376.910000,13800.000000,2.233607e+08,1.0


In [9]:
order_payment['is_fraud_a'] = order_payment['order_id'].isin(satish_fraud['Flo OrderID'])
order_payment['is_fraud_b'] = order_payment['order_id'].isin(satish_fraud['Appetize OrderId'])

order_payment['is_fraud_c'] = order_payment['device_order_id'].isin(satish_fraud['Flo OrderID'])
order_payment['is_fraud_d'] = order_payment['device_order_id'].isin(satish_fraud['Appetize OrderId'])

order_payment['final_fraud'] = order_payment[['is_fraud_a', 'is_fraud_b', 'is_fraud_c', 'is_fraud_d']].any(axis=1)
order_payment = order_payment.drop(columns = {"is_fraud_a", "is_fraud_b",'is_fraud_c', 'is_fraud_d'})

In [10]:
order_payment[order_payment['final_fraud'] == True].compute().shape[0]

2737

In [ ]:
# upload_path = "s3://otg-prod-fraud-data/xxxx"

# order_payment.to_parquet(upload_path, index=False)

In [1]:
import pandas as pd

In [20]:
order_payment.compare(our_df)

order_id           cust_prof_id       original_order_id        \
           self     other         self other              self other   
0      40008556  40008556          NaN   NaN               NaN   NaN   
1      40008566  40008566          NaN   NaN               NaN   NaN   
2      40008578  40008578          NaN   NaN               NaN   NaN   
3      40008585  40008585          NaN   NaN               NaN   NaN   
4      40008590  40008590          NaN   NaN               NaN   NaN   
...         ...       ...          ...   ...               ...   ...   
61721  40083981  40083981          NaN   NaN               NaN   NaN   
61722  40083975  40083975          NaN   NaN               NaN   NaN   
61723  40083979  40083979          NaN   NaN               NaN   NaN   
61724  40083980  40083980          NaN   NaN               NaN   NaN   
61725  40151165  40151165          NaN   NaN               NaN   NaN   

      employer_id       voucher_number        
             self other           self other  
0             NaN   NaN            NaN   NaN  
1             NaN   NaN            NaN   NaN  
2             NaN   NaN            NaN   NaN  
3             NaN   NaN            NaN   NaN  
4             NaN   NaN            NaN   NaN  
...           ...   ...            ...   ...  
61721         NaN   NaN            NaN   NaN  
61722         NaN   NaN            NaN   NaN  
61723         NaN   NaN            NaN   NaN  
61724         NaN   NaN            NaN   NaN  
61725         NaN   NaN            NaN   NaN  

[61726 rows x 10 columns]

In [23]:
# Assuming our_df and order_payment are your two DataFrames

# Columns in our_df but not in order_payment
extra_columns_in_our_df = our_df.columns.difference(order_payment.columns)

print("Extra columns in our_df:", extra_columns_in_our_df)


Extra columns in our_df: Index([], dtype='object')


In [24]:
if set(our_df.columns) == set(order_payment.columns):
    print("Both DataFrames have the same columns (regardless of order).")
else:
    print("The columns are different.")


Both DataFrames have the same columns (regardless of order).


In [25]:
our_df.columns

Index(['bus_date', 'order_id', 'device_order_id', 'time_zone',
       'order_local_time', 'sales_hr', 'ord_beg_time', 'ord_close_time',
       'item_count', 'gross_total', 'net_total', 'taxes', 'exclusive_tax',
       'inclusive_tax', 'reduction_amt', 'cust_prof_id', 'mobile_ord_fl',
       'original_order_id', 'vendor_loc_id', 'fee_amt', 'tip_amount',
       'sync_status', 'menu_vendor_id', 'refund_vendor_id', 'refund_venue_id',
       'pos_terminal_id', 'reduction_cd', 'order_tab_id', 'order_status_cd',
       'united_account_id', 'cust_first_name', 'cust_last_name', 'birth_date',
       'activity_status_code', 'employee_role_name', 'cust_emply_verified_fl',
       'cust_emply_active_fl', 'employer_id', 'employer_name',
       'employer_active_fl', 'default_email_address', 'default_phone_number',
       'company_id', 'primary_email_address', 'primary_email_opt_in_fl',
       'primary_email_active_fl', 'primary_email_verified_fl',
       'extra_ph_number', 'pos_venue_id', 'src_sys_id'

In [27]:
order_payment.columns

Index(['bus_date', 'order_id', 'device_order_id', 'time_zone',
       'order_local_time', 'sales_hr', 'ord_beg_time', 'ord_close_time',
       'item_count', 'gross_total', 'net_total', 'taxes', 'exclusive_tax',
       'inclusive_tax', 'reduction_amt', 'cust_prof_id', 'mobile_ord_fl',
       'original_order_id', 'vendor_loc_id', 'fee_amt', 'tip_amount',
       'sync_status', 'menu_vendor_id', 'refund_vendor_id', 'refund_venue_id',
       'pos_terminal_id', 'reduction_cd', 'order_tab_id', 'order_status_cd',
       'united_account_id', 'cust_first_name', 'cust_last_name', 'birth_date',
       'activity_status_code', 'employee_role_name', 'cust_emply_verified_fl',
       'cust_emply_active_fl', 'employer_id', 'employer_name',
       'employer_active_fl', 'default_email_address', 'default_phone_number',
       'company_id', 'primary_email_address', 'primary_email_opt_in_fl',
       'primary_email_active_fl', 'primary_email_verified_fl',
       'extra_ph_number', 'pos_venue_id', 'src_sys_id'

In [33]:
our_df.columns==order_payment.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [34]:
our_df.shape[1]

62

In [35]:
order_payment.shape[1]

62

In [38]:
len(our_df.columns)

62

In [39]:
len(order_payment.columns)

62